In [ ]:
import os
import re
from pathlib import Path

from einops import rearrange
import numpy as np
from PIL import Image

In [ ]:
import torch

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
from violet.utils.dataloaders import listfiles, MultichannelImageDataset, multichannel_image_dataloader
from violet.utils.dino_utils import DataAugmentationDINOMultichannel
from violet.utils.multichannel import create_pseudocolor_image

In [ ]:
img_dir = '/home/estorrs/violet/data/codex/tile_test_panc_07062021_256res'
img_dir = '/home/estorrs/violet/data/imc/bodenmiller_2019_128res/'


In [ ]:

transform = DataAugmentationDINOMultichannel(
        (0.4, 1.),
        (0.05, 0.4),
        8
)
d1 = MultichannelImageDataset(img_dir, transform=transform, pad=False)
d2 = MultichannelImageDataset(img_dir, transform=None, pad=False)


# sampler = torch.utils.data.DistributedSampler(dataset, shuffle=True)
# data_loader = torch.utils.data.DataLoader(
#     dataset,
#     sampler=sampler,
#     batch_size=64,
# )

In [ ]:
d1.samples, d1.channels

In [ ]:
for i in range(2000, 3000, 1):
    print(i)
    x = rearrange(d2[i], 'c h w -> h w c').numpy()
    pseudo = create_pseudocolor_image(x, d1, ['pan Cytokeratin', 'EGFR', 'Ki-67'])
    plt.imshow(pseudo)
    plt.show()

In [ ]:
xs1 = d1[41]
raw = d2[41]

In [ ]:
import matplotlib.pyplot as plt
for i, x in enumerate(xs1):
    print(i)
    x = rearrange(x, 'c h w -> h w c').numpy()
    pseudo = create_pseudocolor_image(x, d1, ['DAPI', 'Ki67', 'CD20', 'CD8', 'pancytok'])
    plt.imshow(pseudo)
    plt.show()

In [ ]:
x = rearrange(raw, 'c h w -> h w c').numpy()
pseudo = create_pseudocolor_image(x, d1, ['DAPI', 'Ki67', 'CD20', 'CD8', 'pancytok'])
plt.imshow(pseudo)
plt.show()

In [ ]:
dataloader = multichannel_image_dataloader(img_dir, shuffle=True, pad=False)

In [ ]:
dataloader.dataset.samples, dataloader.dataset.channels

In [ ]:
[dataloader.dataset.channels.index(c) for c in ['DAPI', 'Ki67', 'CD20', 'CD8', 'pancytok']]

In [ ]:
import seaborn as sns
from einops import rearrange
def create_pseudocolor_image(img, channels, colors=None):
    if colors is None:
        colors = sns.color_palette()
    
    new = np.zeros((img.shape[0], img.shape[1], 3))
    for i, c in enumerate(channels):
        new += np.repeat(np.expand_dims(img[:, :, c], axis=-1), 3, axis=-1) * np.asarray(colors[i])
    
    new = (new / np.max(new)) * 255.
    return new.astype(np.uint8)

    

In [ ]:
b = next(iter(dataloader))
b.shape

In [ ]:
for i, x in enumerate(b):
    print(i)
    x = rearrange(x, 'c h w -> h w c').numpy()
    pseudo = create_pseudocolor_image(x, [12, 14, 2, 11, 18])
    plt.imshow(pseudo)
    plt.show()

In [ ]:
test = b[39]

In [ ]:
transform = DataAugmentationDINOMultichannel((0.4, 1.), (0.05, 0.4), 8)

In [ ]:
new = transform(test)

In [ ]:
for x in new:
    x = rearrange(x, 'c h w -> h w c').numpy()
    pseudo = create_pseudocolor_image(x, [12, 14, 2, 11, 18])
    plt.imshow(pseudo)
    plt.show()

In [ ]:
transform = DataAugmentationDINOMultichannel(
        (0.4, 1.), (0.05, 0.4), 8
)
dataset = MultichannelImageDataset(
    img_dir, transform=transform, pad=False)

In [ ]:
import torch
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=64,
    shuffle=True,
    drop_last=True,
)

In [ ]:
b = next(iter(dataloader))
len(b)

In [ ]:
b[0].shape